In [45]:
import ee

import geemap
# from sar_change_detection.settings import env, BASE_DIR
import os
import json
from PIL import Image
import json

# Initialize Earth Engine
service_account = "django@sih-sar-prathameshks2003.iam.gserviceaccount.com"
credentials = ee.ServiceAccountCredentials(service_account, "service-account-key.json")
ee.Initialize(credentials)



In [ ]:

def detect_changes():
    if True:
        # try :
        if True:
            # Parse input parameters

            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-07-01"
            end_date = "2024-09-28"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            # Load Sentinel-2 imagery
            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(aoi_geo) \
                .filterDate(start_date, end_date) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

            # Get start and end images
            s2_start = s2_collection.filterDate(start_date, ee.Date(start_date).advance(1, 'month')).median()
            s2_end = s2_collection.filterDate(ee.Date(end_date).advance(-1, 'month'), end_date).median()

            # Perform change detection
            def detect_change(current, previous):
                previous_img = ee.Image(previous)
                diff = current.select(['B4', 'B3', 'B2']).subtract(previous_img.select(['B4', 'B3', 'B2'])).abs()
                return previous_img.addBands(diff.reduce(ee.Reducer.sum()).rename('change'))  # Return the modified image

            first = s2_collection.sort('system:time_start').first()

            changes_obj = ee.List(s2_collection.sort('system:time_start').iterate(detect_change, first))

            print(changes_obj)

            # changes_obj = s2_collection.sort('system:time_start').iterate(detect_change, first)
            changes = ee.ImageCollection.fromImages(changes_obj)

            # changes = s2_collection.sort('system:time_start').map(lambda image: detect_change(image, first))

            # changes = ee.ImageCollection(s2_collection.sort('system:time_start').iterate(detect_change, first))

            change_image = changes.select('change').max()

            print('Change image:', change_image.getInfo())

            # Apply threshold to identify significant changes
            threshold = 0.01  # Adjust this value based on your needs
            significant_changes = change_image.gt(threshold)


            # Print some information to check if significant_changes is empty
            print('Significant changes count:', significant_changes.bandNames().size().getInfo())

            # Convert changes to vectors
            vectors = significant_changes.reduceToVectors(
                scale=10,
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs
            start_image_url = s2_start.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            end_image_url = s2_end.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            change_image_url = significant_changes.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(vectors)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson
            }

            return response

print(detect_changes())

In [46]:
def get_closest_images(s2_collection, start_date, end_date):
    """
    Gets the closest non-None Sentinel-2 images BEFORE the specified start and end dates.

    Args:
        s2_collection: An ee.ImageCollection of Sentinel-2 images.
        start_date: The start date of the image search.
        end_date: The end date of the image search.

    Returns:
        A list containing the closest non-None Sentinel-2 images BEFORE the start and end dates.
    """

    start_images = s2_collection.filterDate('1900-01-01', start_date).sort('system:time_start', False)
    end_images = s2_collection.filterDate('1900-01-01', end_date).sort('system:time_start', False)

    closest_start_image = None
    closest_end_image = None

    # Iterate to find the first non-None image for the start date
    start_list = start_images.toList(start_images.size()) # convert the collection to a list
    for i in range(start_images.size().getInfo()):
        img = ee.Image(start_list.get(i)) # get the image from the list using index i
        if img.getInfo() is not None:
            closest_start_image = img
            break

    # Iterate to find the first non-None image for the end date
    end_list = end_images.toList(end_images.size())
    for i in range(end_images.size().getInfo()):
        img = ee.Image(end_list.get(i))
        if img.getInfo() is not None:
            closest_end_image = img
            break

    return [closest_start_image, closest_end_image]

In [52]:


def detect_changes2():
    if True:
        # try :
        if True:
            # Parse input parameters

            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-06-01"
            end_date = "2024-05-08"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(aoi_geo) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

            # Get the closest images
            # start_date = ee.Date('2024-09-01')  # Replace with your start date
            # end_date = ee.Date('2024-09-28')  # Replace with your end date
            images = get_closest_images(s2_collection, start_date, end_date)

            # Access the images (these won't be None even if exact dates aren't available)
            s2_start = images[0]
            s2_end = images[1]

            start_image_date = s2_start.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
            end_image_date = s2_end.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

            # Subset bands before calculation (similar to the example you provided)
            s2_start_bands = s2_start.select(['B4', 'B3', 'B2'])
            s2_end_bands = s2_end.select(['B4', 'B3', 'B2'])

            # Calculate difference image
            diff = s2_end_bands.subtract(s2_start_bands).abs() # abs() after the subtraction
            change_image = diff.reduce(ee.Reducer.sum()).rename('change')

            print('Change image:', change_image.getInfo())

            # Apply threshold to identify significant changes
            threshold = 0.6  # Adjust this value based on your needs
            significant_changes = change_image.gt(threshold)

            # Print some information to check if significant_changes is empty
            print('Significant changes count:', significant_changes.bandNames().size().getInfo())

            # Convert changes to vectors
            vectors = significant_changes.reduceToVectors(
                geometry=aoi_geo,
                scale=10,
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs
            start_image_url = s2_start.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            end_image_url = s2_end.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            change_image_url = significant_changes.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(vectors)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson,
                'start_date': start_image_date,
                'end_date': end_image_date,
                'bounds' : aoi_geo.bounds().getInfo(),
                'are_images_available': start_image_date != end_image_date
            }

            return response
ch = detect_changes2()


Change image: {'type': 'Image', 'bands': [{'id': 'change', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:32644', 'crs_transform': [10, 0, 600000, 0, -10, 2500020]}]}
Significant changes count: 1
'start_image' : 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/07f199a8b0e4a6536d9392a9a6c1ca92-a6f82397da65274b455d56a5976d5e39:getPixels'
'end_image' : 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/3324de4b1a280bc321e66252f63b58bc-634c2448de0be1f23f9a13083a03ff39:getPixels'
'change_image' : 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/3d414f9fcf8d91237ae24d5695053027-1e40cf7858750e4ca0b538d94821e06b:getPixels'
'changes_geojson' : '{'type': 'FeatureCollection', 'columns': {'count': 'Long<0, 4294967295>', 'label': 'Byte<0, 1>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[82.13436931633424, 2

In [51]:

for k in ch.keys():
    if(k == 'changes_geojson'):
        print(f"'{k}' : {ch[k]},")
    else:
        print(f"'{k}' : '{ch[k]}',")

'start_image' : 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/d9ca3650cc9bb81f8dfa00e199488d84-75c293210111bdebdc76b67a5f40f8a2:getPixels',
'end_image' : 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/3bbec56d91c3f3acb5b4883bde516121-21b2ef1a6e0f5f87181965f3ee10e8ff:getPixels',
'change_image' : 'https://earthengine.googleapis.com/v1/projects/earthengine-legacy/thumbnails/938736a42876bfcd98b4216344922204-458b48be4ba055f051bca5b9b3b07936:getPixels',
'changes_geojson' : {'type': 'FeatureCollection', 'columns': {'count': 'Long<0, 4294967295>', 'label': 'Byte<0, 1>', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[82.13436931633424, 22.189324478416093], [82.13446630616885, 22.18932380290995], [82.13445760192494, 22.188239893352513], [82.14182876238527, 22.18818838863434], [82.14182949242813, 22.18827871420921], [82.15424403729492, 22.18819121539898]

In [54]:
# show images start end and chnages

geemap.show_image(ch['start_image'])
geemap.show_image(ch['end_image'])
geemap.show_image(ch['change_image'])

Output()

Output()

Output()

In [65]:
def detect_changes3():
    if True:
        # try :
        if True:
            # Parse input parameters

            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-06-01"
            end_date = "2024-05-08"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            classifier = ee.Classifier.load('USGS/NLCD/NLCD2016')

            # Get closest non-cloudy Sentinel-2 images
            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                            .filterBounds(aoi_geo) \
                            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
            images = get_closest_images(s2_collection, start_date, end_date)

            # Access the images (these won't be None even if exact dates aren't available)
            s2_start = images[0]
            s2_end = images[1]

            start_image_date = s2_start.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
            end_image_date = s2_end.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

            # Classify start and end images
            s2_start_classified = images[0].classify(classifier)
            s2_end_classified = images[1].classify(classifier)

            # Identify changed pixels based on classification difference
            change_mask = s2_start_classified.neq(s2_end_classified)

             # Convert change mask to polygons
            change_polygons = change_mask.reduceToVectors(
                geometry=aoi_geo,
                scale=30,  # Adjust scale as needed
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs
            start_image_url = s2_start.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            end_image_url = s2_end.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            change_image_url = change_mask.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(change_polygons)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson,
                'start_date': start_image_date,
                'end_date': end_image_date,
                'bounds' : aoi_geo.bounds().getInfo(),
                'are_images_available': start_image_date != end_image_date
            }

            return response



In [66]:
c3 = detect_changes3()



geemap.show_image(c3['start_image'])
geemap.show_image(c3['end_image'])
geemap.show_image(c3['change_image'])

EEException: Asset 'USGS/NLCD/NLCD2016' is not a Classifier.

In [68]:


def detect_changes4():
    if True:
        # try :
        if True:
            # Parse input parameters

            aoi = {"type":"FeatureCollection","features":[{"type":"Feature","properties":{},"geometry":{"type":"Polygon","coordinates":[[[82.134443,22.188232],[82.134443,22.224849],[82.222284,22.224849],[82.222284,22.188232],[82.134443,22.188232]]]}}]}
            start_date =  "2024-06-01"
            end_date = "2024-05-08"
            # aoi_geo = geemap.geojson_to_ee(aoi)
            aoi_geo = geemap.geojson_to_ee(aoi).geometry()  # Convert to Geometry

            s2_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                .filterBounds(aoi_geo) \
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

            # Get the closest images
            # start_date = ee.Date('2024-09-01')  # Replace with your start date
            # end_date = ee.Date('2024-09-28')  # Replace with your end date
            images = get_closest_images(s2_collection, start_date, end_date)

            # Access the images (these won't be None even if exact dates aren't available)
            s2_start = images[0]
            s2_end = images[1]

            start_image_date = s2_start.date().format('YYYY-MM-dd HH:mm:ss').getInfo()
            end_image_date = s2_end.date().format('YYYY-MM-dd HH:mm:ss').getInfo()

            # Cloud masking (example using a simple threshold)
            cloudMask = s2_start.select('QA60').bitwiseAnd(1 << 4).eq(0)
            s2_start = s2_start.updateMask(cloudMask)
            s2_end = s2_end.updateMask(cloudMask)

            # Normalization (example using min-max scaling)
            minVal = ee.Number(0)
            maxVal = ee.Number(10000)
            s2_start = s2_start.divide(maxVal).subtract(minVal).multiply(255).uint8()
            s2_end = s2_end.divide(maxVal).subtract(minVal).multiply(255).uint8()


            # Calculate difference image
            diff = s2_end.subtract(s2_start).abs() # abs() after the subtraction
            # change_image = diff.reduce(ee.Reducer.sum()).rename('change')


            # Apply threshold to identify significant changes
            threshold = 0.6  # Adjust this value based on your needs
            significant_changes = diff.gt(threshold)

            # Print some information to check if significant_changes is empty
            print('Significant changes count:', significant_changes.bandNames().size().getInfo())

            # Visualization (example using a colormap)
            colormap = ['000000', 'FF0000', '00FF00']  # Black, Red, Green
            change_image = diff.updateMask(significant_changes).visualize({
                'min': -threshold,
                'max': threshold,
                'palette': colormap
            })
            # Convert changes to vectors
            vectors = significant_changes.reduceToVectors(
                geometry=aoi_geo,
                scale=10,
                geometryType='polygon',
                maxPixels=1e9,
                eightConnected=False
            )

            # Prepare outputs
            start_image_url = s2_start.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            end_image_url = s2_end.getThumbURL({
                'min': 0,
                'max': 3000,
                'dimensions': 1024,
                'region': aoi_geo,
                'bands': ['B4', 'B3', 'B2']
            })

            change_image_url = significant_changes.getThumbURL({
                'min': 0,
                'max': 1,
                'dimensions': 1024,
                'region': aoi_geo,
                'palette': ['black', 'white']
            })

            # Get GeoJSON of changes
            geojson = geemap.ee_to_geojson(vectors)

            # Prepare the response
            response = {
                'start_image': start_image_url,
                'end_image': end_image_url,
                'change_image': change_image_url,
                'changes_geojson': geojson,
                'start_date': start_image_date,
                'end_date': end_image_date,
                'bounds' : aoi_geo.bounds().getInfo(),
                'are_images_available': start_image_date != end_image_date
            }

            return response
ch4 = detect_changes4()




Significant changes count: 26


EEException: Image.visualize: Cannot provide a palette when visualizing more than one band.

In [ ]:
print(ch4)

geemap.show_image(ch4['start_image'])
geemap.show_image(ch4['end_image'])
geemap.show_image(ch4['change_image'])